## Probabilistic Bayesian Neural Networks

### Introduction

Taking a probabilistic approach to deep learning allows to account for uncertainty, so that models can assign less levels of confidence to incorrect predictions. Sources of uncertainty can be found in the data, due to measurement error or noise in the labels, or the model, due to insufficient data availability for the model to learn effectively.

### The dataset

We use the Wine Quality dataset, which is available in the TensorFlow Datasets. We use the red wine subset, which contains 4,898 examples. The dataset has 11numerical physicochemical features of the wine, and the task is to predict the wine quality, which is a score between 0 and 10. In this example, we treat this as a regression task.

In [ ]:
# install Tensorflow datasets
!pip install tensorflow-datasets

### Setup


In [1]:
import numpy as np
import tensorflow as tf
import tf_keras as keras
import tf_keras.layers as layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

2024-09-14 00:11:16.455463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 00:11:16.477061: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 00:11:16.483701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 00:11:16.499243: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-14 00:11:17.763997: W tensorflow/compiler/tf2

### Create training and evaluation datasets

Here, we load the `wine_quality` dataset using `tfds.load()`, and we convert the target feature to float. Then, we shuffle the dataset and split it into training and test sets. We take the first train_size examples as the train split, and the rest as the test split.

In [2]:
def train_test_splits(dataset_size, train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because the dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    train_dataset = dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [3]:
# Implement experiment pipeline
def run_experiment(model, loss, train_dataset, test_dataset, learning_rate=0.001, num_epochs=100):
    
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate), 
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

### Experiment 1: Standard Neural Network

In [5]:
def create_baseline_model(hidden_units):
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is a single node.
    outputs = layers.Dense(1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
dataset_size = 4898
batch_size = 256
train_size = int(0.85 * dataset_size)
train_dataset, test_dataset = train_test_splits(dataset_size, train_size, batch_size)

2024-09-14 00:11:36.403333: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31060 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:07:00.0, compute capability: 7.0


In [7]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model(hidden_units=[8, 8])
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset, num_epochs=num_epochs)

Start training the model...
Epoch 1/100


2024-09-14 00:11:39.378546: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-09-14 00:11:39.380697: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
I0000 00:00:1726272700.256201 3991382 service.cc:146] XLA service 0x7f49ba584910 initialized for platform CUDA (this does not guarantee that XLA will be

17/17 [==============================] - 3s 33ms/step - loss: 36.1992 - root_mean_squared_error: 6.0166 - val_loss: 36.7424 - val_root_mean_squared_error: 6.0616
Epoch 2/100
17/17 [==============================] - 0s 8ms/step - loss: 34.0892 - root_mean_squared_error: 5.8386 - val_loss: 34.1398 - val_root_mean_squared_error: 5.8429
Epoch 3/100
17/17 [==============================] - 0s 8ms/step - loss: 32.3073 - root_mean_squared_error: 5.6840 - val_loss: 32.0340 - val_root_mean_squared_error: 5.6599
Epoch 4/100
17/17 [==============================] - 0s 8ms/step - loss: 30.6043 - root_mean_squared_error: 5.5321 - val_loss: 30.1221 - val_root_mean_squared_error: 5.4884
Epoch 5/100
17/17 [==============================] - 0s 8ms/step - loss: 28.9555 - root_mean_squared_error: 5.3810 - val_loss: 28.2983 - val_root_mean_squared_error: 5.3196
Epoch 6/100
17/17 [==============================] - 0s 8ms/step - loss: 27.3618 - root_mean_squared_error: 5.2309 - val_loss: 26.5819 - val_root_

We take a sample from the test set use the model to obtain predictions for them. Note that since the baseline model is deterministic, we get a single a point estimate prediction for each test example, with no information about the uncertainty of the model nor the prediction.

In [8]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[0]
predicted = baseline_model.predict(examples)
for idx in range(sample):
    print(f"Predicted: {predicted[idx][0]:.2f}, Actual: {targets[idx].numpy():.2f}")

2024-09-14 00:11:59.571498: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 [==============================] - 0s 190ms/step
Predicted: 6.06, Actual: 6.00
Predicted: 6.23, Actual: 5.00
Predicted: 5.41, Actual: 5.00
Predicted: 5.39, Actual: 6.00
Predicted: 4.67, Actual: 4.00
Predicted: 5.61, Actual: 6.00
Predicted: 5.38, Actual: 5.00
Predicted: 5.47, Actual: 6.00
Predicted: 5.96, Actual: 6.00
Predicted: 6.36, Actual: 7.00


#### Experiment 2: Bayesian neural network (BNN)

The object of the Bayesian approach for modeling neural networks is to capture the epistemic uncertainty, which is uncertainty about the model fitness, due to limited training data.

The idea is that, instead of learning specific weight (and bias) values in the neural network, the Bayesian approach learns weight distributions - from which we can sample to produce an output for a given input - to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights, and the training process is to learn the parameters of these distributions.

In [9]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model

# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

We use the tfp.layers.DenseVariational layer instead of the standard keras.layers.Dense layer in the neural network model.

In [10]:
def create_bnn_model(train_size, hidden_units):
    inputs = create_model_inputs()
    features = layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

The epistemic uncertainty can be reduced as we increase the size of the training data. That is, the more data the BNN model sees, the more it is certain about its estimates for the weights (distribution parameters). Let's test this behaviour by training the BNN model on a small subset of the training set, and then on the full training set, to compare the output variances.

#### Train BNN with a small training subset

In [11]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)
bnn_small_model = create_bnn_model(train_sample_size, hidden_units=[8, 8])
run_experiment(bnn_small_model, mse_loss, small_train_dataset, test_dataset, num_epochs=num_epochs)

Start training the model...
Epoch 1/500
      1/Unknown - 5s 5s/step - loss: 40.7340 - root_mean_squared_error: 6.3815

2024-09-14 00:30:08.660670: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


5/5 [==============================] - 6s 273ms/step - loss: 39.7586 - root_mean_squared_error: 6.3044 - val_loss: 38.4657 - val_root_mean_squared_error: 6.2009
Epoch 2/500
5/5 [==============================] - 0s 21ms/step - loss: 42.8104 - root_mean_squared_error: 6.5421 - val_loss: 36.2894 - val_root_mean_squared_error: 6.0227
Epoch 3/500
5/5 [==============================] - 0s 18ms/step - loss: 40.9981 - root_mean_squared_error: 6.4020 - val_loss: 34.2479 - val_root_mean_squared_error: 5.8512
Epoch 4/500
5/5 [==============================] - 0s 19ms/step - loss: 38.3909 - root_mean_squared_error: 6.1949 - val_loss: 36.6285 - val_root_mean_squared_error: 6.0506
Epoch 5/500
5/5 [==============================] - 0s 20ms/step - loss: 34.3619 - root_mean_squared_error: 5.8607 - val_loss: 41.4321 - val_root_mean_squared_error: 6.4358
Epoch 6/500
5/5 [==============================] - 0s 18ms/step - loss: 41.0321 - root_mean_squared_error: 6.4047 - val_loss: 37.4483 - val_root_mean_s

2024-09-14 00:31:10.553928: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


Since we have trained a BNN model, the model produces a different output each time we call it with the same input, since each time a new set of weights are sampled from the distributions to construct the network and produce an output. The less certain the mode weights are, the more variability (wider range) we will see in the outputs of the same inputs.

In [13]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_small_model)

Predictions mean: 6.09, min: 5.68, max: 6.36, range: 0.68 - Actual: 6.0
Predictions mean: 6.24, min: 5.75, max: 6.41, range: 0.66 - Actual: 5.0
Predictions mean: 5.32, min: 4.6, max: 6.12, range: 1.52 - Actual: 5.0
Predictions mean: 5.44, min: 4.91, max: 6.05, range: 1.14 - Actual: 6.0
Predictions mean: 4.89, min: 4.35, max: 5.62, range: 1.28 - Actual: 4.0
Predictions mean: 5.62, min: 4.93, max: 6.13, range: 1.19 - Actual: 6.0
Predictions mean: 5.64, min: 4.96, max: 6.16, range: 1.2 - Actual: 5.0
Predictions mean: 5.52, min: 4.9, max: 6.12, range: 1.22 - Actual: 6.0
Predictions mean: 6.04, min: 5.56, max: 6.34, range: 0.79 - Actual: 6.0
Predictions mean: 6.28, min: 5.86, max: 6.45, range: 0.58 - Actual: 7.0


#### Train BNN with whole training set

In [14]:
num_epochs = 500
bnn_full_model = create_bnn_model(train_size, hidden_units=[8, 8])
run_experiment(bnn_full_model, mse_loss, train_dataset, test_dataset, num_epochs=num_epochs)

compute_predictions(bnn_full_model)

Start training the model...
Epoch 1/500
17/17 [==============================] - 4s 64ms/step - loss: 31.4879 - root_mean_squared_error: 5.6110 - val_loss: 33.6017 - val_root_mean_squared_error: 5.7963
Epoch 2/500
17/17 [==============================] - 0s 10ms/step - loss: 29.9629 - root_mean_squared_error: 5.4734 - val_loss: 23.4214 - val_root_mean_squared_error: 4.8391
Epoch 3/500
17/17 [==============================] - 0s 10ms/step - loss: 29.8991 - root_mean_squared_error: 5.4677 - val_loss: 31.3225 - val_root_mean_squared_error: 5.5962
Epoch 4/500
17/17 [==============================] - 0s 10ms/step - loss: 25.4590 - root_mean_squared_error: 5.0453 - val_loss: 28.5676 - val_root_mean_squared_error: 5.3444
Epoch 5/500
17/17 [==============================] - 0s 10ms/step - loss: 26.0281 - root_mean_squared_error: 5.1013 - val_loss: 26.7284 - val_root_mean_squared_error: 5.1696
Epoch 6/500
17/17 [==============================] - 0s 9ms/step - loss: 23.0367 - root_mean_squared_e

### Experiment 3: probabilistic Bayesian neural network

So far, the output of the standard and the Bayesian NN models that we built is deterministic, that is, produces a point estimate as a prediction for a given example. We can create a probabilistic NN by letting the model output a distribution. In this case, the model captures the aleatoric uncertainty as well, which is due to irreducible noise in the data, or to the stochastic nature of the process generating the data.

In this example, we model the output as a IndependentNormal distribution, with learnable mean and variance parameters. If the task was classification, we would have used IndependentBernoulli with binary classes, and OneHotCategorical with multiple classes, to model distribution of the model output.

In [15]:
def create_probabilistic_bnn_model(train_size, hidden_units):
    inputs = create_model_inputs()
    features = layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

Since the output of the model is a distribution, rather than a point estimate, we use the negative loglikelihood as our loss function to compute how likely to see the true data (targets) from the estimated distribution produced by the model.

In [16]:
def negative_log_likelihood(targets, predicted_distributions):
    return -predicted_distributions.log_prob(targets)

num_epochs = 1000
probabilistic_bnn_model = create_probabilistic_bnn_model(train_size, hidden_units=[8, 8])
run_experiment(probabilistic_bnn_model, negative_log_likelihood, train_dataset, test_dataset, num_epochs=num_epochs)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 5s 70ms/step - loss: 120.6823 - root_mean_squared_error: 7.0565 - val_loss: 70.3875 - val_root_mean_squared_error: 7.0160
Epoch 2/1000
17/17 [==============================] - 0s 11ms/step - loss: 113.7465 - root_mean_squared_error: 6.9440 - val_loss: 61.2803 - val_root_mean_squared_error: 6.9968
Epoch 3/1000
17/17 [==============================] - 0s 10ms/step - loss: 67.8053 - root_mean_squared_error: 6.9455 - val_loss: 64.5496 - val_root_mean_squared_error: 7.0088
Epoch 4/1000
17/17 [==============================] - 0s 11ms/step - loss: 50.4749 - root_mean_squared_error: 6.9366 - val_loss: 58.3072 - val_root_mean_squared_error: 6.4169
Epoch 5/1000
17/17 [==============================] - 0s 11ms/step - loss: 65.0011 - root_mean_squared_error: 6.8928 - val_loss: 67.3009 - val_root_mean_squared_error: 6.4867
Epoch 6/1000
17/17 [==============================] - 0s 11ms/step - loss: 51.2932 - root_mean_

Now let's produce an output from the model given the test examples. The output is now a distribution, and we can use its mean and variance to compute the confidence intervals (CI) of the prediction.

In [17]:
prediction_distribution = probabilistic_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.98, stddev: 0.77, 95% CI: [7.49 - 4.48] - Actual: 6.0
Prediction mean: 6.26, stddev: 0.74, 95% CI: [7.7 - 4.82] - Actual: 5.0
Prediction mean: 5.08, stddev: 0.63, 95% CI: [6.32 - 3.85] - Actual: 5.0
Prediction mean: 5.11, stddev: 0.63, 95% CI: [6.35 - 3.87] - Actual: 6.0
Prediction mean: 5.04, stddev: 0.65, 95% CI: [6.3 - 3.77] - Actual: 4.0
Prediction mean: 5.34, stddev: 0.64, 95% CI: [6.61 - 4.08] - Actual: 6.0
Prediction mean: 5.28, stddev: 0.64, 95% CI: [6.54 - 4.02] - Actual: 5.0
Prediction mean: 5.47, stddev: 0.65, 95% CI: [6.74 - 4.19] - Actual: 6.0
Prediction mean: 5.83, stddev: 0.74, 95% CI: [7.28 - 4.38] - Actual: 6.0
Prediction mean: 6.31, stddev: 0.76, 95% CI: [7.8 - 4.83] - Actual: 7.0
